In [2]:
import numpy as np
import cv2
import json
import random
import time

import torch
from torch.autograd import Variable
from torch import optim

label_to_idx = {
    'i2':0, 'i4':1, 'i5':2, 'io':3, 'ip':4, 
    'p11':5, 'p23':6, 'p26':7, 'p5':8, 'pl30':9, 
    'pl40':10, 'pl5':11, 'pl50':12, 'pl60':13, 'pl80':14, 
    'pn':15, 'pne':16, 'po':17, 'w57':18
}

def load_data():
    with open("train.json", "r") as f:
        dic = json.loads(f.read())
    names = list(dic)
    labels = list(dic.values())
    num_examples = len(names) // 5
    r = random.random
    random.seed(0)
    random.shuffle(names, random = r)
    random.seed(0)
    random.shuffle(labels, random = r)
    features = []
    idx_labels = []
    for i in range(num_examples):
        name = names[i]
        label = labels[i]
        path = "Train\\" + label + "\\" + name
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)/255.
        img = cv2.resize(img,(128,128))
        features.append(img)
        idx_labels.append(label_to_idx[label])
    trX = features[:int(num_examples*0.8)]
    teX = features[int(num_examples*0.8):]
    trY = idx_labels[:int(num_examples*0.8)]
    teY = idx_labels[int(num_examples*0.8):]
    return trX, teX, trY, teY

class ConvNet(torch.nn.Module):
    def __init__(self, output_dim):
        super(ConvNet, self).__init__()
        
        self.conv = torch.nn.Sequential()
        self.conv.add_module("conv_1", torch.nn.Conv2d(1,10,kernel_size=5))
        self.conv.add_module("BN_1", torch.nn.BatchNorm2d(10))
        self.conv.add_module("relu_1",torch.nn.ReLU())
        self.conv.add_module("dropout",torch.nn.Dropout())
        self.conv.add_module("maxpool_1", torch.nn.MaxPool2d(kernel_size=5))
        
        self.fc = torch.nn.Sequential()
        self.fc.add_module("Linear_l", torch.nn.Linear(10*24*24, 50))
        self.fc.add_module("relu_2", torch.nn.ReLU())
        self.fc.add_module("Linear_2", torch.nn.Linear(50, output_dim))
        
    def forward(self,x):
        x = self.conv.forward(x)
        x = x.view(-1, 10*24*24)
        return self.fc.forward(x)
    
def train(model, loss, optimizer, x_val, y_val):
    x = Variable(x_val, requires_grad = False)
    y = Variable(y_val, requires_grad = False)
    
    optimizer.zero_grad()
    
    fx = model.forward(x)
    output = loss.forward(fx, y.long())
    
    output.backward()
    
    optimizer.step()
    
    return output.item()

def predict(model, x_val):
    x = Variable(x_val, requires_grad = False)
    output = model.forward(x)
    return output.cpu().data.numpy().argmax(axis=1)

torch.manual_seed(42)
trX, teX, trY, teY = load_data()
trX = np.asarray(trX,dtype=np.float32).reshape(-1,1,128,128)
teX = np.asarray(teX,dtype=np.float32).reshape(-1,1,128,128)
trY = np.asarray(trY,dtype=np.int16)
teY = np.asarray(teY,dtype=np.int16)

trX = torch.tensor(trX,dtype=torch.float32)
teX = torch.tensor(teX,dtype=torch.float32)
trY = torch.tensor(trY,dtype=torch.float32)

n_examples = len(trX)
n_classes = 19
model = ConvNet(output_dim = n_classes)

Loss = torch.nn.CrossEntropyLoss(reduction='mean')
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.8)
batch_size = 256
epochs = 50
for i in range(epochs):
    stime = time.time()
    loss = 0.
    num_batches = n_examples // batch_size
    for k in range(num_batches):
        start, end = k * batch_size, (k + 1) * batch_size
        loss += train(model, Loss, optimizer, trX[start:end],trY[start:end])
    predY = predict(model, teX)
    print("Epoch %d, loss = %f, acc = %.2f%%, time %.2f sec"%(i+1, loss/num_batches, 100.*np.mean(predY == teY), time.time()-stime))

Epoch 1, loss = 2.456035, acc = 42.14%, time 23.90 sec
Epoch 2, loss = 1.702502, acc = 59.59%, time 23.53 sec
Epoch 3, loss = 1.222232, acc = 62.52%, time 23.38 sec
Epoch 4, loss = 0.957613, acc = 67.70%, time 24.98 sec
Epoch 5, loss = 0.822490, acc = 72.54%, time 24.14 sec
Epoch 6, loss = 0.659962, acc = 78.07%, time 23.81 sec
Epoch 7, loss = 0.546465, acc = 80.14%, time 24.43 sec
Epoch 8, loss = 0.447074, acc = 79.10%, time 25.01 sec
Epoch 9, loss = 0.384577, acc = 78.07%, time 24.33 sec
Epoch 10, loss = 0.334083, acc = 83.25%, time 25.81 sec
Epoch 11, loss = 0.290758, acc = 84.63%, time 26.03 sec
Epoch 12, loss = 0.250080, acc = 85.15%, time 25.73 sec
Epoch 13, loss = 0.219686, acc = 84.46%, time 24.72 sec
Epoch 14, loss = 0.201304, acc = 84.63%, time 23.33 sec
Epoch 15, loss = 0.180486, acc = 84.46%, time 27.61 sec
Epoch 16, loss = 0.161734, acc = 84.97%, time 24.67 sec
Epoch 17, loss = 0.149005, acc = 85.32%, time 26.41 sec
Epoch 18, loss = 0.134128, acc = 86.18%, time 26.75 sec
E